In [1]:
from adapt.intent import IntentBuilder
from adapt.engine import IntentDeterminationEngine

from functools import wraps

In [2]:
engine = IntentDeterminationEngine()

In [3]:
hide_keyword = ['hide']
for hk in hide_keyword:
    engine.register_entity(hk, "HideKeyword")

hide_intent = IntentBuilder("HideIntent").require("HideKeyword").build()


In [4]:
class IntentCaller:
    """
    This class keeps a mapping between intents and methods.
    """
    def __init__(self, engine=None):
        if not engine:
            engine = IntentDeterminationEngine()
         
        self.engine = engine
        self.intentmapper = {}
        
    def get_function(self, message):
        """
        Query the Intent Engine to find the callable associated with the message.
        """
        print(self.engine, message)
        for intent in self.engine.determine_intent(message):
            print("int", type(intent), intent)
            if intent.get('confidence') > 0:
                func = self.intentmapper.get(intent.get('intent_type'), None)
                print(func)
                if func:
                    return func()
                else:
                    raise ValueError("No function associated with intent {}".format(intent.get('intent_type')))

    
    def register_intent(self, intent):
        """
        Store a function as an intent
        """
        self.engine.register_intent_parser(intent)
        def decorator(func):
            self.intentmapper[intent.name] = func
            return func
        return decorator
        
        
        

In [5]:
intenter = IntentCaller()

In [6]:
@intenter.register_intent(intent=hide_intent)
def callback():
    print("Hello Bob")

In [7]:
intenter.get_function("Hello Bob")

<adapt.engine.IntentDeterminationEngine object at 0x7f057c40af98> Hello Bob
